In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from scipy import stats
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([ lh, rh])

actions = np.array(['accident', 'ambulance', 'breathe','emergency', 'fire', 'help'])

colors = [(245,117,16), (117,245,16), (16,117,245),(255,0,0),(255,140,0),(0,0,0)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [2]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5
model = load_model('ISL.h5')

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    print(predictions)
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
accident
<class 'mediapipe.python.solution_base.SolutionOutputs'>
accident
<class 'mediapipe.python.solution_base.SolutionOutputs'>
accident
<class 'mediapipe.python.solution_base.SolutionOutputs'>
accident
<class 'mediapipe.python.solution_base.SolutionOutputs'>
accident
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ambulance


In [ ]:
cap.release()
cv2.destroyAllWindows()

In [4]:
print(predictions)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
print(res)

[9.7458118e-01 1.9043576e-04 3.5110563e-14 6.4322499e-11 2.5228459e-02
 5.9492709e-17]


In [7]:
print(actions)

['accident' 'ambulance' 'breathe' 'emergency' 'fire' 'help']


In [8]:
print(sequence)

[array([ 8.80297780e-01,  9.56715643e-01, -2.28894251e-07,  8.63069415e-01,
        8.50994468e-01, -5.03742555e-03,  8.05661440e-01,  7.42575526e-01,
       -1.16410078e-02,  7.60766506e-01,  6.72853827e-01, -1.78231187e-02,
        7.46394873e-01,  6.08604133e-01, -2.48695873e-02,  7.54023194e-01,
        7.57930100e-01, -2.59166397e-02,  6.56699359e-01,  7.82103777e-01,
       -3.89327407e-02,  6.66492343e-01,  8.01772833e-01, -4.32695970e-02,
        6.90327585e-01,  8.00854087e-01, -4.53397967e-02,  7.43805587e-01,
        8.18574905e-01, -3.18358950e-02,  6.45533383e-01,  8.36085498e-01,
       -3.92100960e-02,  6.64319813e-01,  8.48233700e-01, -3.63426246e-02,
        6.93195105e-01,  8.50194633e-01, -3.77325863e-02,  7.39540100e-01,
        8.82440031e-01, -3.73044088e-02,  6.46398365e-01,  8.84669423e-01,
       -4.79890183e-02,  6.64964616e-01,  8.93576622e-01, -3.90161797e-02,
        6.93128049e-01,  8.96194339e-01, -3.44740376e-02,  7.36494899e-01,
        9.43807364e-01, 

In [9]:
print(sentence)

['breathe', 'accident']
